# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
import graphlab
import math
import string

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to chen.jenhau@gmail.com and will expire on July 15, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1553451199.log


Now, let us see a preview of what the dataset looks like.

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_punctuation = products['review'].apply(remove_punctuation)
# the column word_count is a dictionary with the word as the key and the frequency of the word as the value
products['word_count'] = graphlab.text_analytics.count_words(review_without_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [6]:
products[269]['word_count']

{'a': 1, 'favorite': 1, 'house': 1, 'in': 1, 'our': 1}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [7]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 3, 'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3, 'ingenious':1, 'love': 2, 'what': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'all': 2,'help': 1, 'cried': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'had': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'fantastic':1, 'help': 1, 'give': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1, 'standarad':1, 'another': 1, 'when': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2, 'nannys': 1,'just': 1, 'food': 1, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [9]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [10]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 12       | 3.000000  | 6.465546     | 0.960964          |

| 6         | 13       | 3.000000  | 7.391976     | 0.975033          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Terminated due to numerical difficulties.

This model may not be ideal. To improve it, consider doing one of the following:
(a) Increasing the regularization.
(b) Standardizing the input data.
(c) Removing highly correlated features.
(d) Removing `inf` and `NaN` values in the training data.

In [11]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 6
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 8.0596

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[mobileupdate]       : 41.9847
word_count[placeid]            : 41.7354
word_count[labelbox]           : 41.151
word_count[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid] : 40.0454
word_count[knobskeeping]       : 36.2091

Lowest Negative Coeffi

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [12]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

In [13]:
weights

name,index,class,value,stderr
(intercept),None,1,1.30337080544,None
word_count,recommend,1,0.303815600015,None
word_count,moist,1,0.671556821414,None
word_count,osocozy,1,0.426326525702,None
word_count,keps,1,7.3963370872,None
word_count,leak,1,-0.24658014554,None
word_count,holder,1,-0.0300523581012,None
word_count,was,1,-0.0530004786379,None
word_count,now,1,0.0383787882079,None
word_count,wipe,1,0.165506649337,None


There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [15]:
num_positive_weights = len(weights[weights['value'] >= 0])
num_negative_weights = len(weights[weights['value'] < 0])

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 68419 
Number of negative weights: 53294 


**Quiz Question:** How many weights are >= 0? `68419`

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [16]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'and': 2, 'all': 1,'love': 1, 'purchased': ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'and': 1, 'would': 2,'almost': 1, 'decals' ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'all': 1, 'money': 1,'into': 1, 'back': 1, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [17]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [18]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [19]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores

[6.7346197270589325, -5.734130996760121, -14.668460404468542]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [50]:
sentiment_model.predict(sample_test_data, output_type='class')

directions = lambda aList: ((int)(anItem/abs(anItem)) for anItem in aList)

sentiments = directions(scores)

for sentiment in sentiments:
    print sentiment

1
-1
-1


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [ ]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data)

**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [55]:
sigmoids = lambda aList: ((1/(1+math.exp(-anItem))) for anItem in aList)

probabilities = sigmoids(scores)

for probability in probabilities:
    print probability


0.998812384838
0.0032232681818
4.26155799666e-07


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [51]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

Class predictions according to GraphLab Create:
[0.9988123848377193, 0.0032232681818011207, 4.2615579966553195e-07]


**Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review? `third`

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [61]:
test_data['probability'] = sentiment_model.predict(test_data, output_type='probability')

In [71]:
test_data.topk('probability', 20)['name']

dtype: str
Rows: 20
['Munchkin Mozart Magic Cube', 'BABYBJORN Potty Chair - Red', 'Safety 1st Tot-Lok Four Lock Assembly', 'Summer Infant Complete Nursery Care Kit', 'Leachco Snoogle Total Body Pillow', 'HALO SleepSack Micro-Fleece Wearable Blanket, Soft Pink, Small', 'Peg Perego Primo Viaggio Car Seat / Infant Carrier with LATCH Base - Black Sable', 'Capri Stroller - Red Tech', 'Wizard Convertible Car Seat with LATCH in Midnight Print', 'Britax Marathon Convertible Car Seat, Granite', 'Britax Decathlon Convertible Car Seat, Tiffany', 'North States Supergate Pressure Mount Clear Choice Wood Gate', 'Fisher-Price Deluxe Jumperoo', "Lilly Gold Sit 'n' Stroll 5 in 1 Car Seat and Stroller Combination, Tuxedo Black (sunshade is not included in the offering)", 'Fisher-Price Rainforest Melodies and Lights Deluxe Gym', 'JP Lizzy Chocolate Ice Classic Tote Set', 'Cloud b Sound Machine Soother, Sleep Sheep', 'Shermag Glider Rocker Combo, Pecan with Oatmeal', 'Traveling Toddler Car Seat Travel Acc

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice] `Britax Decathlon Convertible Car Seat, Tiffany`


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [74]:
test_data.topk('probability', 20, reverse=True)

name,review,rating,word_count,sentiment
Jolly Jumper Arctic SneakA Peek Infant Car Seat ...,"I am a ""research-aholic""in general and have ...",5.0,"{'raining': 1, 'all': 8,'reviewers': 1, 'child': ...",1
Levana Safe N'See DigitalVideo Baby Monitor with ...,This is the first reviewI have ever written out ...,1.0,"{'all': 2, 'they': 4,'just': 1, 'being': 1, ...",-1
Snuza Portable BabyMovement Monitor ...,I would have given theproduct 4 stars for w ...,1.0,"{'contacted': 1, 'being':1, 'money': 1, 'lack' ...",-1
Fisher-Price OceanWonders Aquarium Bouncer ...,We have not had ANY luckwith Fisher-Price ...,2.0,"{'fishstarfish': 1,'infant': 2, 'managed': ...",-1
VTech Communications Safe&amp; Sounds Full Color ...,"This is my second videomonitoring system, the ...",1.0,"{'all': 4, 'reviewers':1, 'beware': 1, ...",-1
Safety 1st High-DefDigital Monitor ...,We bought this babymonitor to replace a ...,1.0,"{'all': 3, 'being': 1,'over': 2, 'it1': 1, ...",-1
Chicco Cortina KeyFit 30Travel System in ...,My wife and I have usedthis system in two cars ...,1.0,"{'all': 4, 'wrestle': 1,'overpriced': 1, 'just': ...",-1
Prince Lionheart WarmiesWipes Warmer ...,*****IMPORTANTUPDATE*****I went to ...,1.0,"{'less': 1, 'move': 1,'not': 4, 'yes': 1, ...",-1
Valco Baby Tri-mode TwinStroller EX- Hot ...,I give one star to thedimension: 1. being 29 ...,1.0,"{'limited': 2, 'forget':1, 'switched': 1, ...",-1
Adiri BPA Free NaturalNurser Ultimate Bottle ...,I will try to write anobjective review of the ...,2.0,"{'all': 2, 'forget': 1,'grateful': 1, 'morei': ...",-1


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice] `The First Years True Choice P400 Premium Digital Monitor, 2 Parent Unit, Peg-Perego Tatamia High Chair, White Latte, Safety 1st High-Def Digital Monitor`

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [77]:
test_data['class'] = sentiment_model.predict(test_data, output_type='class')

true_labels = test_data[test_data['class'] == test_data['sentiment']]

In [79]:
true_labels

name,review,rating,word_count,sentiment
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1, 'standarad':1, 'another': 1, 'when': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2, 'nannys': 1,'just': 1, 'food': 1, ...",1
Nature's Lullabies SecondYear Sticker Calendar ...,"I had a hard time findinga second year calendar, ...",5.0,"{'and': 3, 'all': 1,'later': 1, 'reference': ...",1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4.0,"{'and': 2, 'because': 1,'just': 1, 'less': 1, ...",1
"Lamaze Peekaboo, I LoveYou ...",My son loved this book asan infant. It was ...,5.0,"{'infant': 1, 'being': 1,'all': 1, 'course': 1, ...",1
"Lamaze Peekaboo, I LoveYou ...",Our baby loves this book& has loved it for a ...,5.0,"{'and': 1, 'own': 1,'it': 3, 'our': 1, 'f ...",1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,I bought two for recentbaby showers! The book ...,5.0,"{'and': 2, 'beautiful':1, 'love': 1, 'elmo': 1, ...",1
Baby's First Year UndatedWall Calendar with ...,I searched high and lowfor a first year cale ...,5.0,"{'remembering': 1, 'and':4, 'year': 1, 'am': 1, ...",1
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'and': 2, 'all': 1,'love': 1, 'purchased': ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'and': 1, 'would': 2,'almost': 1, 'decals' ...",-1


In [87]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    data['class'] = model.predict(data, output_type='class')
        
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    true_label_count = len(data[data['class'] == data['sentiment']])

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    
    return true_label_count*1.0/len(data)

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [88]:
accuracy = get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

In [90]:
print "%3.2f" % accuracy

0.91


**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76). `0.91`

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better? `No`

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [91]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [92]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [93]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [94]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [95]:
print train_data[0]['word_count']

{'and': 3, 'love': 1, 'it': 3, 'highly': 1, 'osocozy': 1, 'bags': 1, 'holder': 1, 'leak': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'early': 1, 'not': 2, 'now': 1, 'disappointed': 1, 'wipe': 1, 'keps': 1, 'wise': 1, 'i': 1, 'planet': 1, 'my': 2, 'came': 1}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [96]:
print train_data[0]['word_count_subset']

{'love': 1, 'disappointed': 1}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [97]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.826350     | 0.866815          |

| 6         | 7        | 1.012753     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 1.0531

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [98]:
accuracy = get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

In [100]:
print "%3.2f" % accuracy

0.87


Now, we will inspect the weights (coefficients) of the **simple_model**:

In [101]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552,0.0120888541331
word_count_subset,disappointed,1,-2.35509250061,0.0504149888557
word_count_subset,love,1,1.36543549368,0.0303546295109
word_count_subset,little,1,0.520628636025,0.0214691475665
word_count_subset,loves,1,1.67727145556,0.0482328275384
word_count_subset,product,1,-0.320555492996,0.0154311321362
word_count_subset,well,1,0.504256746398,0.021381300631
word_count_subset,great,1,0.94469126948,0.0209509926591
word_count_subset,easy,1,1.19366189833,0.029288869202
word_count_subset,work,1,-0.621700012425,0.0230330597946


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [102]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`? `10`

In [110]:
coefs_simple = simple_model.coefficients.sort('value', ascending=False)

def significant_words(coefficients, positive=True):
    words = {}
    
    for coef in coefficients:
        if not (coef['name'] == '(intercept)'):
            if positive and coef['value'] > 0:
                words[coef['index']] = coef['value']
            elif not positive and coef['value'] < 0:
                words[coef['index']] = coef['value']
    return words

words_simple = significant_words(coefs_simple)

print words_simple.keys(), len(words_simple.keys())

coefs_sentiment = sentiment_model.coefficients.sort('value', ascending=False)

words_sentiment = significant_words(coefs_sentiment)

print words_sentiment.keys(), len(words_sentiment.keys())

['perfect', 'great', 'love', 'loves', 'able', 'well', 'little', 'easy', 'car', 'old'] 10
['fawn', 'qualitymuch', 'musicallyinclined', 'woodi', 'touchingbiting', '34grandmas', 'orangeis', 'gag', 'woods', 'spiders', 'pinkpig', 'considered1', 'padsthis', 'describedcatches', 'washablemirror', 'nuasence', 'hawwed', 'prazo', 'swimmy', 'roomdarker', 'acurately', 'scold', 'afantastic', 'igual', 'originality', 'transportcompared', 'cartruck', 'unnecessarily', 'loopits', 'frigerated', 'xodo', 'comfortablebaby', 'dragonthemed', '34goose', 'bocks', 'passsengers', 'properlybe', 'assrmble', 'discribed', 'daycarewhoopsother', 'crankyonly', 'pigment', 'handpumpbut', 'pinkprincessthe', 'storethank', 'preganancy', '34eyeballing34', 'divierte', 'uncoordinated', 'handfree', 'attachesremoves', 'toysthat', 'wooded', 'spacy', 'bassett', 'broiler', 'priceyhighly', 'wooden', 'easilywe', '15600the', 'abilitys', 'broiled', 'remoisten', 'circuitry', '34have', 'stereotypical', 'zzzzzz', 'perfectioni', '34accident'

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**? `Yes`

In [113]:
for key in words_simple.keys():
    if not (key in words_sentiment):
        print "Mismatch", key

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [114]:
accuracy_sentiment = get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])
accuracy_simple = get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

print accuracy_sentiment, accuracy_simple

0.979440247047 0.866815074654


Now, compute the classification accuracy of the **simple_model** on the **train_data**:

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set? `sentiment_model`

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [115]:
accuracy_sentiment = get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])
accuracy_simple = get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [116]:
print accuracy_sentiment, accuracy_simple

0.914536837053 0.869300455964


**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set? `sentiment_model`

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [117]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


In [120]:
num_positive  = (test_data['sentiment'] == +1).sum()
num_negative = (test_data['sentiment'] == -1).sum()
print num_positive
print num_negative

print "%3.2f" % (num_positive*1.0/(num_negative+num_positive))

28095
5241
0.84


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76). `0.84`

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)? `Yes`